In [1]:
from bs4 import BeautifulSoup
import requests
import re
import csv
import pandas as pd
import json
import time

In [2]:
try:
    num=0
    url = "https://www.imdb.com/search/title/?title_type=feature&sort=num_votes,desc&start=01"
#     url="https://www.imdb.com/search/title/?title_type=feature&start=9901&ref_=adv_prv"
    while num < 200:
        
        time.sleep(1) # Sleep for 1 seconds
        
        response = requests.get(url)
        response.raise_for_status()
        rsoup = BeautifulSoup(response.text, 'lxml')        
        movies = rsoup.find('div', class_="lister-list").find_all('div',class_='lister-item mode-advanced')
        
        for movie in movies:
            flist = {}
            
            rank_obj = movie.find('h3',class_='lister-item-header')
            if rank_obj is None:
                flist['rank'] = ""
            else:
                flist['rank'] = rank_obj.span.text.strip('.')
            
#             print(rank_obj.span.text.strip('.'))    
                
            title_obj = movie.find('h3',class_='lister-item-header')
            if title_obj is None:
                flist['title_id'] = ""
            else:
                flist['title_id'] = title_obj.a.attrs.get('href')
                
            cast_url = 'https://www.imdb.com' + title_obj.a.attrs.get('href') + 'fullcredits/?ref_=tt_cl_sm'
            movie_id = title_obj.a.attrs.get('href')
            
            name_obj = movie.find('h3',class_='lister-item-header')
            if name_obj is None:
                flist['movie_name'] = ""
            else:
                flist['movie_name'] = name_obj.a.text
                
            year_obj = movie.find('span',class_='lister-item-year text-muted unbold')
            if year_obj is None:
                flist['movie_year'] = ""
            else:
                flist['movie_year'] = year_obj.text

            cert_obj = movie.find('span',class_='certificate')
            if cert_obj is None:
                flist['movie_certificate'] = ""
            else:
                flist['movie_certificate'] = cert_obj.text

            run_obj = movie.find('span',class_='runtime')
            if run_obj is None:
                flist['movie_runtime'] = ""
            else:
                flist['movie_runtime'] = run_obj.text
                
            gen_obj = movie.find('span',class_='genre')
            if gen_obj is None:
                flist['movie_genre'] = ""
            else:
                flist['movie_genre'] = gen_obj.text.strip()

            rat_obj = movie.find('div',class_='inline-block ratings-imdb-rating')
            if rat_obj is None:
                flist['movie_rating'] = ""
            else:
                flist['movie_rating'] = rat_obj.strong.text                

            metastor_obj = movie.find('span',class_=re.compile("metascore"))
            if metastor_obj is None:
                flist['movie_metascore'] = ""
            else:
                flist['movie_metascore'] = metastor_obj.text.strip()

            vote_obj = movie.find('p',class_="sort-num_votes-visible")
            if vote_obj is None:
                flist['movie_votes'] = ""
            else:
                flist['movie_votes'] = vote_obj.text.split()[1]

            gross_obj = movie.find('p',class_="sort-num_votes-visible")
            if gross_obj is None:
                flist['movie_gross'] = ""
            else:
                if gross_obj.text.split()[-1][-1] != 'M' :
                    flist['movie_gross'] = ""
                else:
                    flist['movie_gross'] = gross_obj.text.split()[-1]

            desc_obj = movie.find('p',class_='text-muted',text=re.compile(""))
            if desc_obj is None:
                flist['movie_description'] = ""
            else:
                flist['movie_description'] = desc_obj.text.strip()

            director_obj = movie.find('p',class_='')
            if director_obj is None:
                flist['movie_director'] = ""
            else:
                flist['movie_director'] = director_obj.text.strip().split('|')[0][10:].strip()
                
            stars_obj = movie.find('p',class_='')
            if stars_obj is None:
                flist['movie_stars'] = ""
            else:
                flist['movie_stars'] = stars_obj.text.strip().split('|')[1].strip()[6:].strip()

            url_obj = rsoup.find(class_="lister-page-next next-page")
            if url_obj is None:
                flist['url'] = ""
            else:
                flist['url'] = "https://www.imdb.com" + url_obj.attrs.get('href') 
            
            url = "https://www.imdb.com"+rsoup.find(class_="lister-page-next next-page").attrs.get('href')
            
            ##### Crawl producer information every movie
            
            time.sleep(1) # Sleep for 1 seconds
            
            try:
                
                p_data = {
                "producer_list": [] }
    
                p_response = requests.get(cast_url)
                p_response.raise_for_status()
                psoup = BeautifulSoup(p_response.text, 'lxml')        
                p_list = psoup.find('div',id="fullcredits_content",class_="header")
                p_final = p_list.find('h4',id='producer')
                
                if p_final is None:
                    flist['Producer'] = ""
                else:
                    prows = p_final.find_next_siblings('table')[0].find_all('tr')
            
                    for tr in prows:
                        td = tr.find_all('td')
                        if len(td)==3:
                            row = [i.text for i in td]
                            p_data["producer_list"].append({
                                re.sub("[^a-zA-Z' ]+", '', row[2]).strip() : re.sub("[^a-zA-Z' ]+", '', row[0]).strip()
                            })
                                
                    flist['Producer'] = p_data['producer_list']
                
                ##### crawl crew information for evry movie
                
                time.sleep(1) # Sleep for 1 seconds
                
                c_data = {
                "crew_list": [] }
                
                cast_list = psoup.find("table", {"class" : "cast_list"})
    
                crows = cast_list.find_all('tr')
               
                for cr in crows:
                    cd = cr.find_all('td')
                    if len(cd)==4:
                        irt2 = [i.text for i in cd]
                        c_data["crew_list"].append({
                            re.sub("[^a-zA-Z' ]+", '', irt2[3]).strip() : re.sub("[^a-zA-Z' ]+", '', irt2[1]).strip()  
                        })
    
                flist['Crew'] = c_data['crew_list']
            
            except:
                flist['Producer'] = ""
                flist['Crew'] = ""
   
            final_list.append(flist)
        
        num = num + 1


#         final_json = json.dumps(final_list)
#         print(final_json)
    df = pd.DataFrame(final_list)
    df.reset_index(drop=True)
    df.set_index('rank', inplace=True)
    df = df.replace(' \n','', regex=True)
    df.to_csv('imdb_crawl.csv',mode='a',header=False)
        
        

except Exception as e:
    pass
    print(e)

In [1]:
# final_list

In [11]:
# df = pd.DataFrame(final_list)
# df.reset_index(drop=True)
# df.set_index('rank', inplace=True)
# df = df.replace(' \n','', regex=True)
# df.to_csv('imdb_crawl.csv',mode='a',header=False)